# Project

## Bechdel Test

In [1]:
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Sinan Bursa

In [2]:
credits = pd.read_csv('data/tmdb_5000_credits.csv')
credits = credits[credits.cast != '[]']


movies = pd.read_csv('data/tmdb_5000_movies.csv')
movies.drop(['homepage', 'keywords','original_language','overview','release_date','spoken_languages', \
             'status','title','tagline','vote_count'\
            ], \
            axis=1, \
            inplace=True \
           )

In [3]:
credits.drop(['title', 'crew'], axis=1, inplace=True)
credits['cast_id'] = credits['cast'].apply(lambda row: list(set(pd.read_json(row)['id'])))
#credits['cast_name'] = credits['cast'].apply(lambda row: list(set(pd.read_json(row)['name'])))
#credits['gender'] = credits['cast'].apply(lambda row: list(set(pd.read_json(row)['gender'])))

In [4]:
discount = credits['cast_id'].apply(pd.Series).stack().value_counts()
discount = list(discount[discount > 4].index.astype(int))
discount[:10]

[2231, 380, 62, 1892, 192, 884, 3896, 887, 85, 2963]

In [5]:
credits['cast_id'] = credits['cast_id'].apply(lambda x: [y for y in x if y in discount])
credits['edges'] = credits['cast_id'].apply(lambda x: list(itertools.combinations(x, 2)))
edges = list(credits['edges'].apply(pd.Series).stack())
edges[0:5]

edges_df = pd.DataFrame(edges)

In [6]:
discarded_movies = set()

for idx, movie in credits.iterrows():
    if len(movie['edges']) == 0:
        discarded_movies.add(movie['movie_id'])

print(len(discarded_movies)) 

627


In [7]:
credits = credits[~credits['movie_id'].isin(discarded_movies)]
credits.head()

,movie_id,cast,cast_id,edges
0,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[30485, 236696, 98215, 42286, 68278, 65731, 42...","[(30485, 236696), (30485, 98215), (30485, 4228..."
1,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[2440, 2441, 2449, 2452, 939, 1709, 1711, 1714...","[(2440, 2441), (2440, 2449), (2440, 2452), (24..."
2,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[139549, 17064, 543530, 27319, 121529, 5309, 2...","[(139549, 17064), (139549, 543530), (139549, 2..."
3,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[21505, 53252, 77335, 535, 1574936, 539, 15805...","[(21505, 53252), (21505, 77335), (21505, 535),..."
4,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[17287, 17419, 2206, 2983, 5293, 21044, 57675,...","[(17287, 17419), (17287, 2206), (17287, 2983),..."


In [8]:
movies['profit'] = movies['revenue']-movies['budget']
movies_credits = movies.merge(credits, left_on='id', right_on='movie_id', how='inner').drop(columns=['movie_id'])

In [9]:
movies_credits = movies_credits[movies_credits.genres != '[]']
movies_credits['genre_id'] = movies_credits['genres'].apply(lambda row: list(set(pd.read_json(row)['id'])))

In [10]:
profit_df = pd.DataFrame(movies_credits.cast_id.tolist(), index=movies_credits.profit).stack().reset_index(name='cast_id')[['cast_id','profit']]
profit_df['cast_id'] = profit_df.cast_id.astype(int)
profit_df = profit_df.groupby('cast_id', as_index=False).mean()
profit_df.set_index('cast_id', inplace=True)
profit_df.head()


,profit
cast_id,
3,1.596215e+08
4,1.594258e+08
6,5.387531e+08
7,4.952879e+08
10,6.340370e+08


In [11]:
ranking_df = pd.DataFrame(movies_credits.cast_id.tolist(), index=movies_credits.vote_average).stack().reset_index(name='cast_id')[['cast_id','vote_average']]
ranking_df['cast_id'] = ranking_df.cast_id.astype(int)
ranking_df = ranking_df.groupby('cast_id', as_index=False).mean()
ranking_df.set_index('cast_id', inplace=True)
ranking_df.head()
     

,vote_average
cast_id,
3,6.634375
4,6.366667
6,7.437500
7,7.340000
10,7.466667


In [12]:
genre = movies_credits[['cast_id', 'genre_id']]
genre.loc[:, 'genre_id_disc'] = genre['genre_id'].apply(lambda x: x[0])

/Users/sinangokce/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/sinangokce/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
genre_df = pd.DataFrame(genre.cast_id.tolist(), index=genre.genre_id_disc).stack().reset_index(name='cast_id')[['cast_id','genre_id_disc']]
most_freq_genre = genre_df.groupby(['cast_id']).agg(lambda x:x.value_counts().index[0])


In [14]:
actors = ranking_df.merge(most_freq_genre, on='cast_id', how='inner')
actors = actors.merge(profit_df, on='cast_id', how='inner')

In [16]:
actors = actors.reset_index()
actors.head()

,cast_id,vote_average,genre_id_disc,profit
0,3,6.634375,18,1.596215e+08
1,4,6.366667,80,1.594258e+08
2,6,7.437500,12,5.387531e+08
3,7,7.340000,16,4.952879e+08
4,10,7.466667,16,6.340370e+08


In [17]:
frames = pd.DataFrame()
new_df = pd.DataFrame()

for idx, film in credits.iterrows():
    cast_df = pd.DataFrame(eval(credits['cast'][idx]))
    cast_df['credits'] = idx
    cast_df = cast_df.drop(['character','order', 'credit_id', 'cast_id'],axis = 1)  
    
    frames = [new_df, cast_df]
    new_df = pd.concat(frames, join = 'outer', ignore_index=True)

In [18]:
nodes_df = new_df['credits'].groupby([new_df.gender, new_df.id, new_df.name]).apply(list).reset_index()
nodes_df = nodes_df[nodes_df.id.isin(discount)]
nodes_df.drop(columns=['credits'], inplace=True)
nodes_df = nodes_df[nodes_df['gender'].isin(['1','2'])]

In [19]:
actors = actors.merge(nodes_df, left_on = 'cast_id', right_on='id', how='inner').drop(columns=['cast_id'])

In [22]:
actors = actors[['name', 'id', 'gender', 'genre_id_disc', 'vote_average', 'profit']]

In [25]:
actors[actors['name']=='Leonardo DiCaprio']

,name,id,gender,genre_id_disc,vote_average,profit
824,Leonardo DiCaprio,6193,2,18,7.072727,2.187665e+08


In [24]:
actors.sort_values(by='profit', ascending=False)

,name,id,gender,genre_id_disc,vote_average,profit
2396,Julene Renee,42286,1,16,6.420000,8.703207e+08
306,Adrian Rawlins,1643,2,12,7.500000,7.644205e+08
3411,Jamie Waylett,956224,2,12,7.483333,7.510103e+08
3163,James Phelps,96851,2,12,7.483333,7.510103e+08
1253,Josh Herdman,11212,2,12,7.483333,7.510103e+08
1218,Bonnie Wright,10991,1,12,7.483333,7.510103e+08
3282,Oliver Phelps,140368,2,12,7.483333,7.510103e+08
3162,Matthew Lewis,96841,2,12,7.500000,6.705724e+08
3372,Devon Murray,234922,2,12,7.414286,6.437231e+08
1992,Peter Mayhew,24343,2,12,7.825000,6.405370e+08


In [ ]:
features = nodes_df.set_index('id').drop('name', axis=1)
features.head()